#TMNIST (Typeface MNIST)

# **Read DataSet**

In [82]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.callbacks import EarlyStopping
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

In [83]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
path = "/content/drive/MyDrive/Colab Notebooks/TMNIST_Data.csv"
data = pd.read_csv(path)

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
print(data['names'].unique())

In [ ]:
data['names'].value_counts()

# Seperate Data to X and Y

In [ ]:
X = data.drop(columns={'names','labels'})
y = data[['labels']]

#Viewing Sample Data

In [ ]:
import matplotlib.pyplot as plt

X_images = X.values.reshape(-1,28,28)
fig,axs = plt.subplots(2,9,figsize=(9,9))
for i in range(18):
    r=i//9
    c=i%9
    axs[r][c].set_xticks([])
    axs[r][c].set_yticks([])
    axs[r][c].imshow(X_images[i])
plt.show()

# Seperating Training and Testing set

In [ ]:
from sklearn.model_selection import train_test_split
# creating training set using 80% of data and test set using 20% of data.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train = X_train.values.reshape(-1,28,28)
X_test = X_test.values.reshape(-1,28,28)

print("X_train shape : {}".format(X_train.shape))

print("X_test shape : {}".format(X_test.shape))



In [ ]:
data['labels'].unique()

In [ ]:
data['labels'].nunique()

In [ ]:
# prepare target
def prepare_targets(y_train, y_test):
	le = LabelEncoder()
	le.fit(y_train)
	y_train_enc = le.transform(y_train)
	y_test_enc = le.transform(y_test)
	return y_train_enc, y_test_enc

In [ ]:
y_train_enc,y_test_enc = prepare_targets(y_train,y_test)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train_final = to_categorical(y_train_enc,94,dtype = 'int')
y_test_final = to_categorical(y_test_enc,94,dtype = 'int')

In [ ]:
y_train_final.shape, y_test_final.shape

#Configuring input features for CNN.

In [ ]:
X_train = X_train.reshape(23920, 28, 28,1)
X_test = X_test.reshape(5980, 28, 28,1)

In [ ]:
X_train.shape, X_test.shape

# Normalizing the Dataset for the Neural Network

In [ ]:
X_train, X_test = X_train/255, X_test/255

# Building Neural Network

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,BatchNormalization
from sklearn.metrics import roc_auc_score, accuracy_score

In [ ]:
cnn = Sequential ()


cnn.add(Conv2D(filters = 256 , kernel_size = (3,3) ,padding = 'same'))
cnn.add(BatchNormalization())
cnn.add(tf.keras.layers.Activation('relu'))
cnn.add(MaxPooling2D(pool_size = (2,2)))
cnn.add(Dropout(0.25))

cnn.add(Flatten())
cnn.add(Dense(350))
cnn.add(BatchNormalization())
cnn.add(tf.keras.layers.Activation('relu'))
cnn.add(Dropout(0.25))
cnn.add(Dense(y_train.nunique()[0], activation = 'softmax'))
cnn.summary()

In [ ]:
cnn.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

# Configure the model and start training

In [ ]:
cnn.fit(X_train, y_train_final, epochs=5, batch_size=2000, verbose=1, validation_split=0.15)

# Model Test after train

In [ ]:
test_results = cnn.evaluate(X_test, y_test_final, verbose=1)
print(f'Testing set results - Accuracy: {test_results[1]}%')

# Model Testing after training with Sklearn

In [ ]:
prediction = cnn.predict(X_test)
prediction = lb.inverse_transform(prediction)
print(f'Testing prediction results - Accuracy: {accuracy_score(y_test, prediction)*100}%')